### Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Installing keras tuner

In [ ]:
! pip install keras-tuner

     |████████████████████████████████| 71kB 5.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=7447ed25a3f24074a2691bbb119af258ded554b4b6c7aba21b1201b6a24e7df7
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15357 sha256=b27dd701f48572fbecf9e6df97ffb43283eccbcbb659a9ae90b84d50263a6c3e
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


### Importing Libraries

In [ ]:
import pickle
import numpy as np
from tensorflow import keras

In [ ]:
X=pickle.load(open('/content/drive/MyDrive/mask without mask/maskdata/maskdata/train/X.pkl', 'rb'))
Y=pickle.load(open('/content/drive/MyDrive/mask without mask/maskdata/maskdata/train/Y.pkl', 'rb'))

### Scaling Data

In [ ]:
X = X/255.0

### Built Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense

In [ ]:
def built_model(hp):
  model = Sequential([
                      Conv2D(
                          filters = hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                          kernel_size = hp.Choice('conv_1_kernel', values=[3,5]),
                          activation = 'relu',
                          kernel_initializer = 'he_normal',
                          input_shape = (100,100,3)
                      ),
                      Conv2D(
                          filters = hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
                          kernel_size = hp.Choice('conv_2_kernel', values=[3,5]),
                          activation = 'relu',
                          kernel_initializer = 'he_normal'
                      ),

                      Flatten(),

                      Dense(units = hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
                            activation='relu', kernel_initializer='he_normal'),
                      Dense(units=1, activation='sigmoid', kernel_initializer='glorot_normal')

  ])
  model.compile(optimizer = keras.optimizers.Adam (hp.Choice('learning_rate', values=[1e-3,1e-4])), 
                loss='binary_crossentropy', metrics = [['accuracy']])
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search = RandomSearch(built_model, objective='val_accuracy', max_trials=5, directory='Output', project_name = 'FaceMaskDetection')

In [ ]:
tuner_search.search(X,Y, epochs=5, validation_split=0.1)

Trial 5 Complete [00h 00m 22s]
val_accuracy: 0.9354838728904724

Best val_accuracy So Far: 0.9838709831237793
Total elapsed time: 00h 03m 56s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model = tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 48)        3648      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 94, 94, 96)        41568     
_________________________________________________________________
flatten (Flatten)            (None, 848256)            0         
_________________________________________________________________
dense (Dense)                (None, 112)               95004784  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 113       
Total params: 95,050,113
Trainable params: 95,050,113
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X,Y, epochs=13, validation_split=0.1, initial_epoch=2)

Epoch 3/13
18/18 [==============================] - 1s 55ms/step - loss: 0.1231 - accuracy: 0.9729 - val_loss: 0.2157 - val_accuracy: 0.9355
Epoch 4/13
18/18 [==============================] - 1s 50ms/step - loss: 0.0715 - accuracy: 0.9838 - val_loss: 0.1541 - val_accuracy: 0.9839
Epoch 5/13
18/18 [==============================] - 1s 51ms/step - loss: 0.0191 - accuracy: 0.9910 - val_loss: 0.2208 - val_accuracy: 0.9839
Epoch 6/13
18/18 [==============================] - 1s 51ms/step - loss: 0.0036 - accuracy: 0.9982 - val_loss: 0.0985 - val_accuracy: 0.9839
Epoch 7/13
18/18 [==============================] - 1s 50ms/step - loss: 0.0114 - accuracy: 0.9964 - val_loss: 0.2207 - val_accuracy: 0.9839
Epoch 8/13
18/18 [==============================] - 1s 51ms/step - loss: 0.0221 - accuracy: 0.9982 - val_loss: 0.2287 - val_accuracy: 0.9839
Epoch 9/13
18/18 [==============================] - 1s 50ms/step - loss: 0.0150 - accuracy: 0.9946 - val_loss: 0.1010 - val_accuracy: 0.9839
Epoch 10/13
1

In [ ]:
model.save('/content/drive/MyDrive/mask without mask/maskdata/maskdata/train/FaceMask.h5')